**Updated**: 05.12.2021

In [ ]:
import numpy as np
import sqlite3 as sq
import matplotlib.pyplot as plt
import matplotlib.collections
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
import json
import matplotlib.text as mtext
from colors import Colors
import seaborn as sns

%matplotlib inline
# green mean, yellow median
#from IPython.core.display import display, HTML
#display(HTML("<style>.container { width:100% !important; }</style>"))

In [ ]:
connection = sq.connect("./performance-measurements/merged-noqlogs-nohops.db")
connection.row_factory = sq.Row

In [ ]:
vantage_points = {
                  'US East':                  {'short': 'us-east_'},
                  'Asia Pacific Northeast':   {'short': 'ap-northeast_'},
                  'Africa South':             {'short': 'af-south_'},
                  'Europe Central':           {'short': 'eu-central_'},
                  'Asia Pacific Southeast':   {'short': 'ap-southeast_'},
                  'US West':                  {'short': 'us-west_'},
                  'South America East':       {'short': 'sa-east_'},
                  'All':                      {'short': ''}
                 }

In [ ]:
# basically quic_drop_dups but also dropping all queries with errors
sql = """
CREATE TABLE IF NOT EXISTS dns_measurements_clean AS
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'us-east_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming
UNION ALL
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'us-west_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming
UNION ALL
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'ap-northeast_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming
UNION ALL
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'af-south_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming
UNION ALL
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'eu-central_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming
UNION ALL
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'ap-southeast_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming
UNION ALL
SELECT * 
FROM (SELECT *, datetime(created, 'unixepoch') as created_date FROM dns_measurements WHERE id LIKE 'sa-east_%' AND r_code IS NOT NULL)
GROUP BY strftime('%d', created_date), strftime('%H', created_date), protocol, ip, cache_warming;
"""
cursor = connection.cursor()
cursor.execute(sql)
connection.commit()
cursor.close()

In [ ]:
def quic_drop_dups(measurements_quic):
    # not used, instead we create a cleaned table
    # drop duplicates on floored timestamp and ip -> this removes resolvers with doubled ports (e.g. adguard)
    measurements_quic['created_datetime'] = pd.to_datetime(measurements_quic['created'], unit='s')
    measurements_quic['created_datetime_hour'] = measurements_quic['created_datetime'].dt.floor('h')
    measurements_quic_cleaned = measurements_quic.drop_duplicates(['ip', 'created_datetime_hour'])
    measurements_quic_cleaned.drop(['created_datetime', 'created_datetime_hour'], axis=1, inplace=True)
    return measurements_quic_cleaned
    
def average(items, key):
    total = 0
    for item in items:
        try:
            total += item[key]
        except:
            pass
    return total/len(items)

def getTraceroutesForProtocol(protocol, vp=''):
    def generateStatement(protocol, vp):
        return "SELECT ip FROM dns_measurements_clean {} WHERE {}.protocol = '{}' AND {}.response_ip IS NOT NULL AND NOT {}.cache_warming AND measurement.id LIKE '{}%'".format(protocol, protocol, protocol, protocol, protocol, vp)
    statements = {
        "udp": generateStatement("udp", vp),
        "tcp": generateStatement("tcp", vp),
        "tls": generateStatement("tls", vp),
        "https": generateStatement("https", vp),
        "quic": generateStatement("quic", vp)
    }
    del statements[protocol]
    sql = """
    SELECT * from traceroutes WHERE (dns_measurement_id, hop_ip, dest_port, protocol) IN (SELECT id, ip, port, protocol FROM dns_measurements_clean measurement
    WHERE measurement.protocol = '{}' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({}));
    """.format(protocol, vp, *statements.values())
    df = pd.read_sql_query(sql, connection)
    return df

def getMeanTraceroutesPairwise(vp=''):
    udp = getTraceroutesForProtocolPairwise("udp", vp)["rtt"].div(1000000)
    tcp = getTraceroutesForProtocolPairwise("tcp", vp)["rtt"].div(1000000)
    tls = getTraceroutesForProtocolPairwise("tls", vp)["rtt"].div(1000000)
    https = getTraceroutesForProtocolPairwise("https", vp)["rtt"].div(1000000)
    quic = getTraceroutesForProtocolPairwise("quic", vp)["rtt"].div(1000000)
    
    fig = plt.figure(figsize = (10, 7))
    ax = fig.add_subplot(111)
    bp = ax.boxplot([udp.values.tolist(), tcp.values.tolist(), tls.values.tolist(), https.values.tolist(), quic.values.tolist()], vert = 0, showfliers = False)

    ax.set_yticklabels(["UDP", "TCP", "TLS", "HTTPS", "QUIC"])

    ax.set_xlabel("ms", fontsize = 12)
    ax.set_ylabel("Protocol", fontsize = 12)
    plt.title("RTT for Hosts with all Protocols Reachable")

    plt.show(bp)
    
    df = pd.DataFrame({ 
        "Protocol": ["UDP", "TCP", "TLS", "HTTPS", "QUIC"],
        "Samples": [len(udp), len(tcp), len(tls), len(https), len(quic)]
    })
    print(df.to_markdown())
    
def getTraceroutesForProtocolPairwise(protocol, vp=''):
    def generateStatement(protocol, vp):
        return "SELECT ip FROM dns_measurements_clean {} WHERE {}.protocol = '{}' AND {}.response_ip IS NOT NULL AND NOT {}.cache_warming".format(protocol, protocol, protocol, protocol, protocol)
    statements = {
        "udp": generateStatement("udp", vp),
        "tcp": generateStatement("tcp", vp),
        "tls": generateStatement("tls", vp),
        "https": generateStatement("https", vp),
        "quic": generateStatement("quic", vp)
    }
    del statements[protocol]
    sql = """
    SELECT * from traceroutes WHERE (dns_measurement_id, hop_ip, dest_port, protocol) IN (SELECT id, ip, port, protocol FROM dns_measurements_clean measurement
    WHERE measurement.protocol = '{}' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND NOT measurement.cache_warming
      AND measurement.id LIKE '{}%'
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})) 
      AND hop_ip IN (SELECT DISTINCT(hop_ip) FROM traceroutes WHERE protocol = 'udp' AND hop_ip = dest_ip)
      AND hop_ip IN (SELECT DISTINCT(hop_ip) FROM traceroutes WHERE protocol = 'quic' AND hop_ip = dest_ip)
      AND hop_ip IN (SELECT DISTINCT(hop_ip) FROM traceroutes WHERE protocol = 'tcp' AND hop_ip = dest_ip)
      AND hop_ip IN (SELECT DISTINCT(hop_ip) FROM traceroutes WHERE protocol = 'tls' AND hop_ip = dest_ip)
      AND hop_ip IN (SELECT DISTINCT(hop_ip) FROM traceroutes WHERE protocol = 'https' AND hop_ip = dest_ip);
    """.format(protocol, vp, *statements.values())
    df = pd.read_sql_query(sql, connection)
    return df

def getMeanTraceroutes(vp=''):
    udp_df = getTraceroutesForProtocolPairwise("udp", vp)
    udp_df["rtt"] = udp_df["rtt"].div(1000000)
    tcp_df = getTraceroutesForProtocolPairwise("tcp", vp)
    tcp_df["rtt"] = tcp_df["rtt"].div(1000000)
    tls_df = getTraceroutesForProtocolPairwise("tls", vp)
    tls_df["rtt"] = tls_df["rtt"].div(1000000)
    https_df = getTraceroutesForProtocolPairwise("https", vp)
    https_df["rtt"] = https_df["rtt"].div(1000000)
    quic_df = getTraceroutesForProtocolPairwise("quic", vp)
    quic_df["rtt"] = quic_df["rtt"].div(1000000)
    
    
    udp = udp_df.groupby("dest_ip")["rtt"].median().reset_index()["rtt"]
    tcp = tcp_df.groupby("dest_ip")["rtt"].median().reset_index()["rtt"]
    tls = tls_df.groupby("dest_ip")["rtt"].median().reset_index()["rtt"]
    https = https_df.groupby("dest_ip")["rtt"].median().reset_index()["rtt"]
    quic = quic_df.groupby("dest_ip")["rtt"].median().reset_index()["rtt"]
    
    return {'UDP': udp, 'TCP': tcp, 'TLS': tls, 'HTTPS': https, 'QUIC': quic}
        
def getQUICMedianTraceroutes(cacheWarming = False, protocol = "quic", vp=''):
    cacheCondition = "AND NOT d.cache_warming" if not cacheWarming else "AND d.cache_warming"
    sql = """
    SELECT
       d.id, d.ip, d.port, d.protocol, d.q_ui_c_handshake_duration as quic_handshake_duration, d.query_time, d.tcp_handshake_duration, d.tls_handshake_duration, d.total_time, d.q_ui_c_version
    FROM dns_measurements_clean d
    WHERE d.protocol = '{}' {}
    AND d.id LIKE '{}%'
    AND d.id IN (SELECT DISTINCT(t.dns_measurement_id) FROM traceroutes t where t.dest_ip = t.hop_ip AND t.protocol = '{}');
    """.format(protocol, cacheCondition, vp, protocol)
    handshake_duration = pd.read_sql_query(sql, connection)
    sql = """
    SELECT
    t.dns_measurement_id as id, dest_port as port, protocol, dest_ip, hop_ip, rtt
    FROM traceroutes t
    WHERE t.protocol = '{}' AND t.dest_ip = t.hop_ip AND id LIKE '{}%'
    """.format(protocol, vp)
    rtts = pd.read_sql_query(sql, connection).groupby("id").sample(n = 1)
    return pd.merge(handshake_duration, rtts, on = ["id", "protocol", "port"])

def getQUICMedianResolveTime(vp=''):
    sql = f"""
    SELECT ip, query_time FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'quic' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{vp}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean udp WHERE udp.protocol = 'udp' AND udp.response_ip IS NOT NULL AND NOT udp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tcp WHERE tcp.protocol = 'tcp' AND tcp.response_ip IS NOT NULL AND NOT tcp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tls WHERE tls.protocol = 'tls' AND tls.response_ip IS NOT NULL AND NOT tls.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean https WHERE https.protocol = 'https' AND https.response_ip IS NOT NULL AND NOT https.cache_warming);
    """
    quic_df = pd.read_sql_query(sql, connection)
    
    quic_df["query_time"] = quic_df["query_time"].div(1000000)
    
    quic = quic_df.groupby("ip")["query_time"].median().reset_index()
    return quic

def getUDP(vp=''):
    sql = f"""
    SELECT * FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'udp' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{vp}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tcp WHERE tcp.protocol = 'tcp' AND tcp.response_ip IS NOT NULL AND NOT tcp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tls WHERE tls.protocol = 'tls' AND tls.response_ip IS NOT NULL AND NOT tls.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean https WHERE https.protocol = 'https' AND https.response_ip IS NOT NULL AND NOT https.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean quic WHERE quic.protocol = 'quic' AND quic.response_ip IS NOT NULL AND NOT quic.cache_warming);
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getTCP(vp=''):
    sql = f"""
    SELECT * FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'tcp' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{vp}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean udp WHERE udp.protocol = 'udp' AND udp.response_ip IS NOT NULL AND NOT udp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tls WHERE tls.protocol = 'tls' AND tls.response_ip IS NOT NULL AND NOT tls.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean https WHERE https.protocol = 'https' AND https.response_ip IS NOT NULL AND NOT https.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean quic WHERE quic.protocol = 'quic' AND quic.response_ip IS NOT NULL AND NOT quic.cache_warming);
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getTLS(vp=''):
    sql = f"""
    SELECT * FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'tls' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{vp}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean udp WHERE udp.protocol = 'udp' AND udp.response_ip IS NOT NULL AND NOT udp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tcp WHERE tcp.protocol = 'tcp' AND tcp.response_ip IS NOT NULL AND NOT tcp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean https WHERE https.protocol = 'https' AND https.response_ip IS NOT NULL AND NOT https.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean quic WHERE quic.protocol = 'quic' AND quic.response_ip IS NOT NULL AND NOT quic.cache_warming);
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getHTTPS(vp=''):
    sql = f"""
    SELECT * FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'https' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{vp}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean udp WHERE udp.protocol = 'udp' AND udp.response_ip IS NOT NULL AND NOT udp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tcp WHERE tcp.protocol = 'tcp' AND tcp.response_ip IS NOT NULL AND NOT tcp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tls WHERE tls.protocol = 'tls' AND tls.response_ip IS NOT NULL AND NOT tls.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean quic WHERE quic.protocol = 'quic' AND quic.response_ip IS NOT NULL AND NOT quic.cache_warming);
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getQUIC(vp=''):
    sql = f"""
    SELECT * FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'quic' AND measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND measurement.id LIKE '{vp}%'
      AND NOT measurement.cache_warming
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean udp WHERE udp.protocol = 'udp' AND udp.response_ip IS NOT NULL AND NOT udp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tcp WHERE tcp.protocol = 'tcp' AND tcp.response_ip IS NOT NULL AND NOT tcp.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean tls WHERE tls.protocol = 'tls' AND tls.response_ip IS NOT NULL AND NOT tls.cache_warming)
      AND measurement.ip IN (SELECT ip FROM dns_measurements_clean https WHERE https.protocol = 'https' AND https.response_ip IS NOT NULL AND NOT https.cache_warming);
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getQUICNegotiatedProtocol(vp=''):
    sql = f"""
    SELECT DISTINCT(measurement.ip), measurement.q_ui_c_negotiated_protocol FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'quic' AND measurement.r_code IS NOT NULL
    AND NOT measurement.cache_warming
    AND measurement.id LIKE '{vp}%';
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getQUICVersions(vp=''):
    sql = f"""
    SELECT DISTINCT(measurement.ip), measurement.q_ui_c_version FROM dns_measurements_clean measurement
    WHERE measurement.protocol = 'quic' AND measurement.r_code IS NOT NULL
    AND NOT measurement.cache_warming
    AND measurement.id LIKE '{vp}%';
    """
    cursor = connection.cursor()
    cursor.execute(sql)
    items = list(cursor.fetchall())
    cursor.close()
    return items

def getNumberOfSamples(vp=''):
    udp = len(getUDP(vp))
    tcp = len(getTCP(vp))
    tls = len(getTLS(vp))
    https = len(getHTTPS(vp))
    quic = len(getQUIC(vp))
    return pd.DataFrame({ 
        "Protocol": ["UDP", "TCP", "TLS", "HTTPS", "QUIC"],
        "Samples": [udp, tcp, tls, https, quic]
    })

def getData(vp=''):
    udp = getUDP(vp)
    tcp = getTCP(vp)
    tls = getTLS(vp)
    https = getHTTPS(vp)
    quic = getQUIC(vp)
    return pd.DataFrame(udp + tcp + tls + https + quic, columns = udp[0].keys())

def calculateCDF(items):
    count, bins_count = np.histogram(items, bins="auto")

    pdf = count / sum(count)

    # using numpy np.cumsum to calculate the CDF
    # We can also find using the PDF values by looping and adding
    cdf = np.cumsum(pdf)
    return bins_count[1:], cdf

def setBoxesWhite(bp):
    for patch in bp['boxes']:
        patch.set_facecolor("white")

### Total Servers used

In [ ]:
for key, vp in vantage_points.items():
    print(key)
    vp['allData'] = getData(vp['short'])
    print(len(vp['allData']["ip"].unique()))

In [ ]:
for key, vp in vantage_points.items():
    vp['UDP'] = getUDP(vp['short'])
    vp['TCP'] = getTCP(vp['short'])
    vp['TLS'] = getTLS(vp['short'])
    vp['HTTPS'] = getHTTPS(vp['short'])
    vp['QUIC'] = getQUIC(vp['short'])

### Average Total Times

In [ ]:
for key, vp in vantage_points.items():
    vp['average_total_time'] = {
        'UDP': average(vp['UDP'], "total_time") / 1000000,
        'TCP': average(vp['TCP'], "total_time") / 1000000,
        'TLS': average(vp['TLS'], "total_time") / 1000000,
        'HTTPS': average(vp['HTTPS'], "total_time") / 1000000,
        'QUIC': average(vp['QUIC'], "total_time") / 1000000
    }

In [ ]:
row_labels = [key for key in vantage_points]
col_labels = ['UDP', 'TCP', 'TLS', 'HTTPS', 'QUIC']
total_times = [[vp['average_total_time'][protocol] for protocol in col_labels] for key, vp in vantage_points.items()]
total_times_df = pd.DataFrame(total_times, columns=col_labels, index=row_labels)
total_times_df.style.background_gradient(cmap='RdYlGn_r')

In [ ]:
total_times_df.style.background_gradient(cmap='RdYlGn_r', axis=1)

In [ ]:
sns.heatmap(total_times_df, annot=True, fmt='g', cmap='Reds')

#### Number of Samples

In [ ]:
for key, vp in vantage_points.items():
    vp['number_of_samples'] = getNumberOfSamples(vp['short']).set_index("Protocol")
total_samples = [[vp['number_of_samples'].loc[protocol]['Samples'] for protocol in col_labels] for key, vp in vantage_points.items()]
total_samples_df = pd.DataFrame(total_samples, columns=col_labels, index=row_labels)
total_samples_df

### Comparing Protocols

In [ ]:
for key, vp in vantage_points.items():
    vp['query_time'] = {
    'UDP': list(map(lambda x: x["query_time"] / 1000000, vp['UDP'])),
    'TCP': list(map(lambda x: x["query_time"] / 1000000, vp['TCP'])),
    'TLS': list(map(lambda x: x["query_time"] / 1000000, vp['TLS'])),
    'HTTPS': list(map(lambda x: x["query_time"] / 1000000, vp['HTTPS'])),
    'QUIC': list(map(lambda x: x["query_time"] / 1000000, vp['QUIC']))
    }

i = 0
fig, ax = plt.subplots(4, 2, figsize=(18, 24))
for key, vp in vantage_points.items():
    qt = vp['query_time']
    bp = ax[int(i/2),i%2].boxplot([qt['UDP'], qt['TCP'], qt['TLS'], qt['HTTPS'], qt['QUIC']], vert = 0, sym = "", patch_artist=True, meanline = True, showmeans = True)
    setBoxesWhite(bp)

    for median in bp["medians"]:
        x, y = median.get_data()
        ax[int(i/2),i%2].text(x[0] - 9, y[0] - 0.2, "{:.0f}".format(x[0]), fontsize = 9, color='black')

    ax[int(i/2),i%2].set_yticklabels(["UDP", "TCP", "TLS", "HTTPS", "QUIC"])
    ax[int(i/2),i%2].set_xlabel('ms', fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("Protocol", fontsize = 12)
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    ax[int(i/2),i%2].grid()
    i += 1

plt.savefig("figures/resolve_time_boxplot.pdf", bbox_inches='tight')
plt.show()

In [ ]:
i = 0
fig, ax = plt.subplots(4, 2, figsize=(16, 24))

for key, vp in vantage_points.items():
    qt = vp['query_time']
    ax[int(i/2),i%2].plot(calculateCDF(qt['UDP'])[0], calculateCDF(qt['UDP'])[1], label="UDP", color = Colors.udp)
    ax[int(i/2),i%2].plot(calculateCDF(qt['TCP'])[0], calculateCDF(qt['TCP'])[1], label="TCP", color = Colors.tcp)
    ax[int(i/2),i%2].plot(calculateCDF(qt['TLS'])[0], calculateCDF(qt['TLS'])[1], label="TLS", color = Colors.tls)
    ax[int(i/2),i%2].plot(calculateCDF(qt['HTTPS'])[0], calculateCDF(qt['HTTPS'])[1], label="HTTPS", color = Colors.https)
    ax[int(i/2),i%2].plot(calculateCDF(qt['QUIC'])[0], calculateCDF(qt['QUIC'])[1], label="QUIC", color = Colors.quic)
    
    ax[int(i/2),i%2].set_xlim(0, 350)
    ax[int(i/2),i%2].set_xlabel("ms", fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("CDF")
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    
    ax[int(i/2),i%2].legend()
    ax[int(i/2),i%2].grid()
    i += 1
    
plt.savefig("figures/resolve_time_cdf.pdf", bbox_inches='tight')
plt.show()

### Average resolve time

In [ ]:
np.set_printoptions(precision=3)
for key, vp in vantage_points.items():
    vp['average_query_time'] = {
        'UDP': round(average(vp['UDP'], "query_time") / 1000000, 2),
        'TCP': round(average(vp['TCP'], "query_time") / 1000000, 2),
        'TLS': round(average(vp['TLS'], "query_time") / 1000000, 2),
        'HTTPS': round(average(vp['HTTPS'], "query_time") / 1000000, 2),
        'QUIC': round(average(vp['QUIC'], "query_time") / 1000000, 2)
    }
    print(key)
    print(vp['average_query_time'])

In [ ]:
for key, vp in vantage_points.items():
    vp['total_time'] = {
    'UDP': list(map(lambda x: x["total_time"] / 1000000, vp['UDP'])),
    'TCP': list(map(lambda x: x["total_time"] / 1000000, vp['TCP'])),
    'TLS': list(map(lambda x: x["total_time"] / 1000000, vp['TLS'])),
    'HTTPS': list(map(lambda x: x["total_time"] / 1000000, vp['HTTPS'])),
    'QUIC': list(map(lambda x: x["total_time"] / 1000000, vp['QUIC']))
    }

i = 0
fig, ax = plt.subplots(4, 2, figsize=(18, 24))
for key, vp in vantage_points.items():
    tt = vp['total_time']
    bp = ax[int(i/2),i%2].boxplot([tt['UDP'], tt['TCP'], tt['TLS'], tt['HTTPS'], tt['QUIC']], vert = 0, sym = "", patch_artist=True, meanline = True, showmeans = True)
    setBoxesWhite(bp)

    for median in bp["medians"]:
        x, y = median.get_data()
        ax[int(i/2),i%2].text(x[0] - 9, y[0] - 0.2, "{:.0f}".format(x[0]), fontsize = 9, color='black')
    
    ax[int(i/2),i%2].set_xlim(0, 1600)
    ax[int(i/2),i%2].set_yticklabels(["UDP", "TCP", "TLS", "HTTPS", "QUIC"])
    ax[int(i/2),i%2].set_xlabel('ms', fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("Protocol", fontsize = 12)
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    ax[int(i/2),i%2].grid()
    i += 1

plt.savefig("figures/total_time_boxplot.pdf", bbox_inches='tight')
plt.show()

In [ ]:
prot_1 = 'UDP'
prot_2 = 'QUIC'

fig, ax = plt.subplots(figsize=(20, 10))
x = []
y = []
z = []
for key, vp in vantage_points.items():
    ttu = vp['total_time'][prot_1]
    x.extend(len(ttu) * [key])
    y.extend(ttu)
    z.extend(len(ttu) * [prot_1])
    
    ttq = vp['total_time'][prot_2]
    x.extend(len(ttq) * [key])
    y.extend(ttq)
    z.extend(len(ttq) * [prot_2])
    
data = pd.DataFrame(dict(x=x, y=y, z=z))

ax.set_title('Total Times Distribution by Vantage Point', fontsize = 20)
ax.set_ylim(0, 1600)

v = sns.violinplot(data=data, x='x', y='y',
               split=True, hue = 'z',
               ax = ax, inner='quartile',
               bw = 0.3)
v.set_xlabel('Vantage Point', fontsize = 18)
v.set_ylabel('ms', fontsize = 18)
v.tick_params(labelsize=13)

plt.legend(title='Protocol', fontsize='x-large', title_fontsize='16')
plt.savefig("figures/total_time_violinplot.pdf", bbox_inches='tight')
plt.show()

In [ ]:
for key, vp in vantage_points.items():
    vp['setup_time'] = {
    'UDP': list(map(lambda x: x["udp_socket_setup_duration"] / 1000000, vp['UDP'])),
    'TCP': list(map(lambda x: x["tcp_handshake_duration"] / 1000000, vp['TCP'])),
    'TLS': list(map(lambda x: (x["tcp_handshake_duration"] + x["tls_handshake_duration"]) / 1000000, vp['TLS'])),
    'HTTPS': list(map(lambda x: (x["tcp_handshake_duration"] + x["tls_handshake_duration"]) / 1000000, vp['HTTPS'])),
    'QUIC': list(map(lambda x: x["q_ui_c_handshake_duration"] / 1000000, vp['QUIC']))
    }

    
i = 0
fig, ax = plt.subplots(4, 2, figsize=(18, 24))
for key, vp in vantage_points.items():
    st = vp['setup_time']
    bp = ax[int(i/2),i%2].boxplot([st['TCP'], st['TLS'], st['HTTPS'], st['QUIC']], vert = 0, showfliers = False, patch_artist=True, meanline = True, showmeans = True)
    setBoxesWhite(bp)

    for median in bp["medians"]:
        x, y = median.get_data()
        ax[int(i/2),i%2].text(x[0] - 20, y[0] - 0.2, "{:.0f}".format(x[0]), fontsize = 9, color='black')

    ax[int(i/2),i%2].set_xlim(0, 1300)
    ax[int(i/2),i%2].set_yticklabels(["TCP", "TLS", "HTTPS", "QUIC"])
    ax[int(i/2),i%2].set_xlabel("ms", fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("Protocol", fontsize = 12)
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    ax[int(i/2),i%2].grid()
    i += 1


plt.savefig("figures/handshake_time_boxplot.pdf", bbox_inches='tight')
plt.show(bp)

In [ ]:
i = 0
fig, ax = plt.subplots(4, 2, figsize=(16, 24))

for key, vp in vantage_points.items():
    tt = vp['total_time']
    ax[int(i/2),i%2].plot(calculateCDF(tt['UDP'])[0], calculateCDF(tt['UDP'])[1], label="UDP", color = Colors.udp)
    ax[int(i/2),i%2].plot(calculateCDF(tt['TCP'])[0], calculateCDF(tt['TCP'])[1], label="TCP", color = Colors.tcp)
    ax[int(i/2),i%2].plot(calculateCDF(tt['TLS'])[0], calculateCDF(tt['TLS'])[1], label="TLS", color = Colors.tls)
    ax[int(i/2),i%2].plot(calculateCDF(tt['HTTPS'])[0], calculateCDF(tt['HTTPS'])[1], label="HTTPS", color = Colors.https)
    ax[int(i/2),i%2].plot(calculateCDF(tt['QUIC'])[0], calculateCDF(tt['QUIC'])[1], label="QUIC", color = Colors.quic)
    
    ax[int(i/2),i%2].set_xlim(0, 1200)
    ax[int(i/2),i%2].set_xlabel("ms", fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("CDF")
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    
    ax[int(i/2),i%2].legend()
    ax[int(i/2),i%2].grid()
    i += 1
    
plt.savefig("figures/total_time_cdf.pdf", bbox_inches='tight')
plt.show()

In [ ]:
i = 0
fig, ax = plt.subplots(4, 2, figsize=(16, 24))

for key, vp in vantage_points.items():
    st = vp['setup_time']
    #ax[int(i/2),i%2].plot(calculateCDF(st['UDP'])[0], calculateCDF(st['UDP'])[1], label="UDP", color = Colors.udp)
    ax[int(i/2),i%2].plot(calculateCDF(st['TCP'])[0], calculateCDF(st['TCP'])[1], label="TCP", color = Colors.tcp)
    ax[int(i/2),i%2].plot(calculateCDF(st['TLS'])[0], calculateCDF(st['TLS'])[1], label="TLS", color = Colors.tls)
    ax[int(i/2),i%2].plot(calculateCDF(st['HTTPS'])[0], calculateCDF(st['HTTPS'])[1], label="HTTPS", color = Colors.https)
    ax[int(i/2),i%2].plot(calculateCDF(st['QUIC'])[0], calculateCDF(st['QUIC'])[1], label="QUIC", color = Colors.quic)
    
    ax[int(i/2),i%2].set_xlim(0, 800)
    ax[int(i/2),i%2].set_xlabel("ms", fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("CDF")
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    
    ax[int(i/2),i%2].legend()
    ax[int(i/2),i%2].grid()
    i += 1
    
plt.savefig("figures/setup_time_cdf.pdf", bbox_inches='tight')
plt.show()

### Traceroutes per Protocol

In [ ]:
for key, vp in vantage_points.items():
    vp['mean_traceroutes'] = getMeanTraceroutes(vp['short'])

i = 0
fig, ax = plt.subplots(4, 2, figsize=(16, 24))
for key, vp in vantage_points.items():
    mt = vp['mean_traceroutes']
    bp = ax[int(i/2),i%2].boxplot([mt['UDP'].values.tolist(), mt['TCP'].values.tolist(), mt['TLS'].values.tolist(), mt['HTTPS'].values.tolist(), mt['QUIC'].values.tolist()], vert = 0, sym = "", patch_artist=True, meanline = True, showmeans = True)
    setBoxesWhite(bp)

    for median in bp["medians"]:
        x, y = median.get_data()
        ax[int(i/2),i%2].text(x[0] - 4, y[0] - 0.2, "{:.0f}".format(x[0]), fontsize = 9, color='black')

    ax[int(i/2),i%2].set_yticklabels(["UDP", "TCP", "TLS", "HTTPS", "QUIC"])
    ax[int(i/2),i%2].set_xlabel("ms", fontsize = 12)
    ax[int(i/2),i%2].set_ylabel("Protocol", fontsize = 12)
    ax[int(i/2),i%2].set_title(key, fontsize = 16)
    ax[int(i/2),i%2].grid()
    i += 1

plt.savefig("figures/median_traceroutes_boxplot.pdf", bbox_inches='tight')
plt.show()

### QUIC Handshake Duration / RTT

In [ ]:
times = getQUICMedianTraceroutes()
times["handshake/rtt"] = times.apply(lambda row: row["quic_handshake_duration"]/row["rtt"], axis = 1)
times["resolve-rtt"] = times.apply(lambda row: (row["query_time"] - row["rtt"]) / 1000000, axis = 1)

In [ ]:
times_uncached = getQUICMedianTraceroutes(cacheWarming = True)
times_uncached["resolve-rtt"] = times_uncached.apply(lambda row: (row["query_time"] - row["rtt"]) / 1000000, axis = 1)

In [ ]:
def plotHandshake():
    def filterNA(df):
        return df[df["handshake/rtt"].notna()]["handshake/rtt"].tolist()
    
    def addTLS(row):
        if row["tcp_handshake_duration"] and row["tls_handshake_duration"]:
            return (row["tcp_handshake_duration"] + row["tls_handshake_duration"])/row["rtt"]
        return np.nan
    
    quicTimes = getQUICMedianTraceroutes(protocol = "quic")
    tcpTimes = getQUICMedianTraceroutes(protocol = "tcp")
    tlsTimes = getQUICMedianTraceroutes(protocol = "tls")
    httpsTimes = getQUICMedianTraceroutes(protocol = "https")
    quicTimes["handshake/rtt"] = quicTimes.apply(lambda row: row["quic_handshake_duration"]/row["rtt"], axis = 1)
    tcpTimes["handshake/rtt"] = tcpTimes.apply(lambda row: row["tcp_handshake_duration"]/row["rtt"], axis = 1)
    tlsTimes["handshake/rtt"] = tlsTimes.apply(addTLS, axis = 1)
    httpsTimes["handshake/rtt"] = httpsTimes.apply(addTLS, axis = 1)
    
    fig = plt.figure(figsize = (10, 7))
    
    distQUIC = filterNA(quicTimes)
    distTCP = filterNA(tcpTimes)
    distTLS = filterNA(tlsTimes)
    distHTTPS = filterNA(httpsTimes)
    
    plt.plot(calculateCDF(distQUIC)[0], calculateCDF(distQUIC)[1], label="QUIC", color = Colors.quic)
    plt.plot(calculateCDF(distTCP)[0], calculateCDF(distTCP)[1], label="TCP", color = Colors.tcp)
    plt.plot(calculateCDF(distTLS)[0], calculateCDF(distTLS)[1], label="TLS", color = Colors.tls)
    plt.plot(calculateCDF(distHTTPS)[0], calculateCDF(distHTTPS)[1], label="HTTPS", color = Colors.https)
    
    plt.xlim([0, 8])
    plt.grid()
    plt.legend()
    plt.xlabel("Round-Trips", fontsize = 12)
    plt.ylabel("CDF")
    
    plt.savefig("figures/handshake-rtt.pdf", bbox_inches='tight')
    plt.show()
    

plotHandshake()

In [ ]:
def getQLogPandas():
    items = []
    with open("qlog-results.json", "r") as f:
        result = json.load(f)
    for ip, ports in result.items():
        for port, stats in ports.items():
            items.append({ "ip": ip, "port": int(port), "success": stats["success"], "retry": stats["retry"], "version_negotiation": stats["version_negotiation"] })
            
    return pd.DataFrame(items)

qlogResult = getQLogPandas()

In [ ]:
quicTimes = getQUICMedianTraceroutes(protocol = "quic")

In [ ]:
combined = pd.merge(quicTimes, qlogResult, on = ["ip", "port"])

In [ ]:
def normalized(row):
    duration = row["quic_handshake_duration"]
    if not row["success"]:
        return duration / row["rtt"]
    if row["retry"]:
        duration -= row["rtt"]
    if row["version_negotiation"]:
        duration -= row["rtt"]
    return duration / row["rtt"]

combined["handshake/rtt"] = combined.apply(lambda row: row["quic_handshake_duration"]/row["rtt"], axis = 1)
combined["n(handshake/rtt)"] = combined.apply(normalized, axis = 1)

In [ ]:
def plotHandshakeNormalized():
    def normalized(row):
        duration = row["quic_handshake_duration"]
        if not row["success"]:
            return duration / row["rtt"]
        if row["retry"]:
            duration -= row["rtt"]
        if row["version_negotiation"]:
            duration -= row["rtt"]
        return duration / row["rtt"]
    
    def filterNA(df):
        return df[df["handshake/rtt"].notna()]["handshake/rtt"].tolist()
    
    def addTLS(row):
        if row["tcp_handshake_duration"] and row["tls_handshake_duration"]:
            return (row["tcp_handshake_duration"] + row["tls_handshake_duration"])/row["rtt"]
        return np.nan
    
    quicTimes = getQUICMedianTraceroutes(protocol = "quic")
    combined = pd.merge(quicTimes, qlogResult, on = ["ip", "port"])
    combined["n(handshake/rtt)"] = combined.apply(normalized, axis = 1)
    tcpTimes = getQUICMedianTraceroutes(protocol = "tcp")
    tlsTimes = getQUICMedianTraceroutes(protocol = "tls")
    httpsTimes = getQUICMedianTraceroutes(protocol = "https")
    quicTimes["handshake/rtt"] = quicTimes.apply(lambda row: row["quic_handshake_duration"]/row["rtt"], axis = 1)
    tcpTimes["handshake/rtt"] = tcpTimes.apply(lambda row: row["tcp_handshake_duration"]/row["rtt"], axis = 1)
    tlsTimes["handshake/rtt"] = tlsTimes.apply(addTLS, axis = 1)
    httpsTimes["handshake/rtt"] = httpsTimes.apply(addTLS, axis = 1)
    
    fig = plt.figure(figsize = (10, 7))
    
    distQUIC = combined[combined["n(handshake/rtt)"].notna()]["n(handshake/rtt)"].tolist()
    distTCP = filterNA(tcpTimes)
    distTLS = filterNA(tlsTimes)
    distHTTPS = filterNA(httpsTimes)
    
    plt.plot(calculateCDF(distQUIC)[0], calculateCDF(distQUIC)[1], label="QUIC", color = Colors.quic)
    plt.plot(calculateCDF(distTCP)[0], calculateCDF(distTCP)[1], label="TCP", color = Colors.tcp)
    plt.plot(calculateCDF(distTLS)[0], calculateCDF(distTLS)[1], label="TLS", color = Colors.tls)
    plt.plot(calculateCDF(distHTTPS)[0], calculateCDF(distHTTPS)[1], label="HTTPS", color = Colors.https)
    
    plt.xlim([0, 8])
    plt.grid()
    plt.legend()
    plt.xlabel("Round-Trips", fontsize = 12)
    plt.ylabel("CDF")
    
    plt.savefig("figures/handshake-rtt-normalized.pdf", bbox_inches='tight')
    plt.show()
    

plotHandshakeNormalized()

In [ ]:
def normalized(row):
    duration = row["quic_handshake_duration"]
    if not row["success"]:
        return duration / row["rtt"]
    if row["retry"]:
        duration -= row["rtt"]
    if row["version_negotiation"]:
        duration -= row["rtt"]
    return duration

quicTimes = getQUICMedianTraceroutes(protocol = "quic")
combined = pd.merge(quicTimes, qlogResult, on = ["ip", "port"])
tcpTimes = getQUICMedianTraceroutes(protocol = "tcp")
tlsTimes = getQUICMedianTraceroutes(protocol = "tls")
httpsTimes = getQUICMedianTraceroutes(protocol = "https")
udpTimes = getQUICMedianTraceroutes(protocol = "udp")

In [ ]:
combined["n(handshake)"] = combined.apply(normalized, axis = 1)
combined["n(total_time)"] = combined["total_time"] - combined["n(handshake)"]

fig = plt.figure(figsize = (10, 7))

distUDP = udpTimes[udpTimes["total_time"].notna()]["total_time"].div(1000000).tolist()
distQUIC = combined[combined["n(total_time)"].notna()]["n(total_time)"].div(1000000).tolist()
distTCP = tcpTimes[tcpTimes["total_time"].notna()]["total_time"].div(1000000).tolist()
distTLS = tlsTimes[tlsTimes["total_time"].notna()]["total_time"].div(1000000).tolist()
distHTTPS = httpsTimes[httpsTimes["total_time"].notna()]["total_time"].div(1000000).tolist()

plt.plot(calculateCDF(distQUIC)[0], calculateCDF(distQUIC)[1], label="QUIC", color = Colors.quic)
plt.plot(calculateCDF(distTCP)[0], calculateCDF(distTCP)[1], label="TCP", color = Colors.tcp)
plt.plot(calculateCDF(distTLS)[0], calculateCDF(distTLS)[1], label="TLS", color = Colors.tls)
plt.plot(calculateCDF(distHTTPS)[0], calculateCDF(distHTTPS)[1], label="HTTPS", color = Colors.https)
plt.plot(calculateCDF(distUDP)[0], calculateCDF(distUDP)[1], label="UDP", color = Colors.udp)

plt.grid()
plt.legend()
plt.xlabel("ms", fontsize = 12)
plt.ylabel("CDF")
plt.xlim([0, 1200])

plt.savefig("figures/total_time_normalized_cdf.pdf", bbox_inches='tight')
plt.show()

In [ ]:
combined["n(handshake)"] = combined.apply(normalized, axis = 1)
combined["n(total_time)"] = combined["total_time"] - combined["n(handshake)"]

distQUIC = combined[combined["n(total_time)"].notna()]["n(total_time)"].div(1000000).tolist()
distTCP = tcpTimes[tcpTimes["total_time"].notna()]["total_time"].div(1000000).tolist()
distTLS = tlsTimes[tlsTimes["total_time"].notna()]["total_time"].div(1000000).tolist()
distHTTPS = httpsTimes[httpsTimes["total_time"].notna()]["total_time"].div(1000000).tolist()
distUDP = udpTimes[udpTimes["total_time"].notna()]["total_time"].div(1000000).tolist()

fig = plt.figure(figsize = (10, 7))
ax = fig.add_subplot(111)
bp = ax.boxplot([distUDP, distTCP, distTLS, distHTTPS, distQUIC], vert = 0, showfliers = False, patch_artist=True, meanline = True, showmeans = True)
setBoxesWhite(bp)

for median in bp["medians"]:
    x, y = median.get_data()
    ax.text(x[0] - 20, y[0] - 0.2, "{:.0f}".format(x[0]), fontsize = 9, color='black')

ax.set_yticklabels(["UDP", "TCP", "TLS", "HTTPS", "QUIC"])

ax.set_xlabel("ms", fontsize = 12)
ax.set_ylabel("Protocol", fontsize = 12)
plt.grid()

plt.savefig("figures/total_time_normalized_boxplot.pdf", bbox_inches='tight')
plt.show(bp)

## Smallest Variance

In [ ]:
def getSetupTime():
    def generateStatement(protocol):
        return "SELECT ip FROM dns_measurements {} WHERE {}.protocol = '{}' AND {}.response_ip IS NOT NULL AND NOT {}.cache_warming".format(protocol, protocol, protocol, protocol, protocol)
    statements = {
        "udp": generateStatement("udp"),
        "tcp": generateStatement("tcp"),
        "tls": generateStatement("tls"),
        "https": generateStatement("https"),
        "quic": generateStatement("quic")
    }
    sql = """
    SELECT ip, protocol, q_ui_c_handshake_duration / 1000000 as quic_handshake_duration, tcp_handshake_duration / 1000000 as tcp_handshake_duration, tls_handshake_duration / 1000000 as tls_handshake_duration, port FROM dns_measurements measurement
    WHERE measurement.r_code IS NOT NULL AND measurement.total_time IS NOT NULL AND measurement.total_time <= 5000000000
      AND NOT measurement.cache_warming
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
      AND measurement.ip IN ({})
    """.format(*statements.values())
    df = pd.read_sql_query(sql, connection)
    return df

setupTime = getSetupTime()
setupTime["tls_total"] = setupTime.apply(lambda row: row["tcp_handshake_duration"] + row["tls_handshake_duration"] if row["protocol"] == "tls" or row["protocol"] == "https" else None, axis = 1)

In [ ]:
smallestVar = setupTime.groupby("ip")["quic_handshake_duration"].var().reset_index(name = "var").nsmallest(50, "var")
largestVar = setupTime.groupby("ip")["quic_handshake_duration"].var().reset_index(name = "var").nlargest(50, "var")

In [ ]:
records = pd.DataFrame([*udp, *tcp, *tls, *https, *quic], columns = udp[0].keys())

In [ ]:
smallestRecords = records[records["ip"].isin(smallestVar["ip"].unique())]
smallestRecords["tls_total"] = smallestRecords.apply(lambda row: row["tcp_handshake_duration"] + row["tls_handshake_duration"] if row["protocol"] == "tls" or row["protocol"] == "https" else None, axis = 1)
largestRecords = records[records["ip"].isin(largestVar["ip"].unique())]
largestRecords["tls_total"] = largestRecords.apply(lambda row: row["tcp_handshake_duration"] + row["tls_handshake_duration"] if row["protocol"] == "tls" or row["protocol"] == "https" else None, axis = 1)

In [ ]:
fig = plt.figure(figsize = (10, 7))
tcpCDF = calculateCDF(smallestRecords[smallestRecords["protocol"] == "tcp"]["tcp_handshake_duration"].div(1000000).tolist())
tlsCDF = calculateCDF(smallestRecords[smallestRecords["protocol"] == "tls"]["tls_total"].div(1000000).tolist())
httpsCDF = calculateCDF(smallestRecords[smallestRecords["protocol"] == "https"]["tls_total"].div(1000000).tolist())
quicCDF = calculateCDF(smallestRecords[smallestRecords["protocol"] == "quic"]["q_ui_c_handshake_duration"].div(1000000).tolist())

plt.plot(tcpCDF[0], tcpCDF[1], label="TCP", color = Colors.tcp)
plt.plot(tlsCDF[0], tlsCDF[1], label="TLS", color = Colors.tls)
plt.plot(httpsCDF[0], httpsCDF[1], label="HTTPS", color = Colors.https)
plt.plot(quicCDF[0], quicCDF[1], label="QUIC", color = Colors.quic)
plt.xlim([0, 600])
plt.ylim([0, 1.04])

plt.xlabel("ms", fontsize = 12)
plt.ylabel("CDF")

plt.legend()
plt.grid()

plt.savefig("figures/setup_time_low_variance_pdf.pdf", bbox_inches='tight')

plt.show()

In [ ]:
fig = plt.figure(figsize = (10, 7))
tcpCDF = calculateCDF(largestRecords[largestRecords["protocol"] == "tcp"]["tcp_handshake_duration"].div(1000000).tolist())
tlsCDF = calculateCDF(largestRecords[largestRecords["protocol"] == "tls"]["tls_total"].div(1000000).tolist())
httpsCDF = calculateCDF(largestRecords[largestRecords["protocol"] == "https"]["tls_total"].div(1000000).tolist())
quicCDF = calculateCDF(largestRecords[largestRecords["protocol"] == "quic"]["q_ui_c_handshake_duration"].div(1000000).tolist())

plt.plot(tcpCDF[0], tcpCDF[1], label="TCP", color = Colors.tcp)
plt.plot(tlsCDF[0], tlsCDF[1], label="TLS", color = Colors.tls)
plt.plot(httpsCDF[0], httpsCDF[1], label="HTTPS", color = Colors.https)
plt.plot(quicCDF[0], quicCDF[1], label="QUIC", color = Colors.quic)
plt.xlim([0, 1200])
plt.ylim([0, 1.04])

plt.xlabel("ms", fontsize = 12)
plt.ylabel("CDF")

plt.legend()
plt.grid()

plt.savefig("figures/setup_time_high_variance_pdf.pdf", bbox_inches='tight')

plt.show()